In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('label.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./google_images\00b42b2c-f193-4863-b92c-0245cb...,159,316,561,653
1,./google_images\03273806-bb1e-48da-8c8b-a0133a...,191,383,291,356
2,./google_images\0369b20e-b432-4409-90f9-242087...,330,464,307,357
3,./google_images\07064c2c-2aa3-4419-91a4-92916d...,194,336,272,350
4,./google_images\074d85b8-42ec-4d17-9b8e-9e51ae...,137,320,157,233


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./google_images\\00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./google_images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./google_images\\00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./google_images\\00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg',
 './google_images\\03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg',
 './google_images\\0369b20e-b432-4409-90f9-2420877aa386___8151536c79159a1557421da5f27f9f0e.jpg.jpeg',
 './google_images\\07064c2c-2aa3-4419-91a4-92916de8e54c___mahindra-scorpio-old-car-500x500.jpg.jpeg',
 './google_images\\074d85b8-42ec-4d17-9b8e-9e51ae060243___hqdefault.jpg.jpeg',
 './google_images\\07aaab79-71ee-4ea3-a9e6-640191183947___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_1208484d1392541449-nissan-terrano-official-review-img_20140215_181708.jpg',
 './google_images\\07bd977e-d578-49a2-b345-7cee5a4db6bf___new_1031520d1356604430-skoda-rapid-joins-family-edit-sold-wp_000281.jpg.jpeg',
 './google_images\\0850c175-0b8d-47f2-801c-e29f1dbdb367___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_571.jpg',
 './google_images\\0a0d1748-48cd-4114-90cb-b5baf0b3cb

In [8]:
file_path = image_path[0]
file_path

'./google_images\\00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg'

In [9]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import cv2
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:

cv2.rectangle(img,(159,561),(316,653),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
label = df.iloc[:,1:].values

In [15]:
label[0]

array([159, 316, 561, 653], dtype=int64)

In [16]:
data=[]
output=[]

for ind in range(len(image_path)):

    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    #preprocessing
    load_image = load_img(image, target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_img_arr = load_image_arr/255.0

    #normalization
    xmin,xmax,ymin,ymax = label[ind]
    nxmin,nxmax= xmin/w , xmax/w
    nymin,nymax = ymin/h , ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)
    
    data.append(norm_load_img_arr)
    output.append(label_norm)

In [17]:
x = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [18]:
x.shape,y.shape

((135, 224, 224, 3), (135, 4))

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((108, 224, 224, 3), (27, 224, 224, 3), (108, 4), (27, 4))

In [20]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable


In [22]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [23]:
inception_resnet = InceptionResNetV2(weights = "imagenet",include_top =False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable = False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation = "relu")(headmodel)
headmodel = Dense(250, activation = "relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs= inception_resnet.input , outputs=headmodel)

In [24]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 25, 25, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 25, 25, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 25, 25, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 25, 25, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_169[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_167 (Batch  (None, 5, 5, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_175 (Batch  (None, 5, 5, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 5, 5, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 custom_scale_layer_35 (CustomS  (None, 5, 5, 2080)  0           ['block8_5_ac[0][0]',            
 caleLayer)                                                       'block8_6_conv[0][0]']          
          

                                                                                                  
 custom_scale_layer_37 (CustomS  (None, 5, 5, 2080)  0           ['block8_7_ac[0][0]',            
 caleLayer)                                                       'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['custom_scale_layer_37[0][0]']  
                                                                                                  
 conv2d_196 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 5, 5, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 500)          19200500    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 250)          125250      ['dense[0][0]']                  
                                                                                                  
 dense_2 (

In [25]:
from tensorflow.keras.callbacks import TensorBoard

In [26]:
tfb = TensorBoard('object_detection')

In [27]:
history = model.fit(x=x_train, y=y_train , batch_size = 10, epochs =100, validation_data =(x_test, y_test),callbacks=[tfb])

Epoch 1/100
11/11 [==============================] - 37s 2s/step - loss: 0.0789 - val_loss: 0.0948
Epoch 2/100
11/11 [==============================] - 22s 2s/step - loss: 0.0696 - val_loss: 0.0712
Epoch 3/100
11/11 [==============================] - 20s 2s/step - loss: 0.0573 - val_loss: 0.0615
Epoch 4/100
11/11 [==============================] - 20s 2s/step - loss: 0.0521 - val_loss: 0.0503
Epoch 5/100
11/11 [==============================] - 20s 2s/step - loss: 0.0409 - val_loss: 0.0425
Epoch 6/100
11/11 [==============================] - 22s 2s/step - loss: 0.0378 - val_loss: 0.0407
Epoch 7/100
11/11 [==============================] - 20s 2s/step - loss: 0.0355 - val_loss: 0.0409
Epoch 8/100
11/11 [==============================] - 20s 2s/step - loss: 0.0346 - val_loss: 0.0389
Epoch 9/100
11/11 [==============================] - 20s 2s/step - loss: 0.0314 - val_loss: 0.0377
Epoch 10/100
11/11 [==============================] - 20s 2s/step - loss: 0.0304 - val_loss: 0.0383
Epoch 11/

11/11 [==============================] - 20s 2s/step - loss: 0.0278 - val_loss: 0.0363
Epoch 84/100
11/11 [==============================] - 20s 2s/step - loss: 0.0278 - val_loss: 0.0360
Epoch 85/100
11/11 [==============================] - 20s 2s/step - loss: 0.0278 - val_loss: 0.0372
Epoch 86/100
11/11 [==============================] - 20s 2s/step - loss: 0.0283 - val_loss: 0.0365
Epoch 87/100
11/11 [==============================] - 20s 2s/step - loss: 0.0284 - val_loss: 0.0365
Epoch 88/100
11/11 [==============================] - 20s 2s/step - loss: 0.0282 - val_loss: 0.0355
Epoch 89/100
11/11 [==============================] - 19s 2s/step - loss: 0.0280 - val_loss: 0.0365
Epoch 90/100
11/11 [==============================] - 20s 2s/step - loss: 0.0281 - val_loss: 0.0361
Epoch 91/100
11/11 [==============================] - 20s 2s/step - loss: 0.0288 - val_loss: 0.0359
Epoch 92/100
11/11 [==============================] - 20s 2s/step - loss: 0.0283 - val_loss: 0.0354
Epoch 93/100


In [28]:
history = model.fit(x=x_train, y=y_train , batch_size = 10, epochs =200, validation_data =(x_test, y_test),callbacks=[tfb], initial_epoch = 100)

Epoch 101/200
11/11 [==============================] - 19s 2s/step - loss: 0.0275 - val_loss: 0.0357
Epoch 102/200
11/11 [==============================] - 19s 2s/step - loss: 0.0277 - val_loss: 0.0355
Epoch 103/200
11/11 [==============================] - 19s 2s/step - loss: 0.0278 - val_loss: 0.0356
Epoch 104/200
11/11 [==============================] - 22s 2s/step - loss: 0.0278 - val_loss: 0.0354
Epoch 105/200
11/11 [==============================] - 19s 2s/step - loss: 0.0277 - val_loss: 0.0357
Epoch 106/200
11/11 [==============================] - 20s 2s/step - loss: 0.0275 - val_loss: 0.0351
Epoch 107/200
11/11 [==============================] - 20s 2s/step - loss: 0.0243 - val_loss: 0.0211
Epoch 108/200
11/11 [==============================] - 19s 2s/step - loss: 0.0113 - val_loss: 0.0104
Epoch 109/200
11/11 [==============================] - 19s 2s/step - loss: 0.0054 - val_loss: 0.0109
Epoch 110/200
11/11 [==============================] - 20s 2s/step - loss: 0.0047 - val_los

11/11 [==============================] - 19s 2s/step - loss: 4.5352e-04 - val_loss: 0.0066
Epoch 180/200
11/11 [==============================] - 19s 2s/step - loss: 4.3552e-04 - val_loss: 0.0063
Epoch 181/200
11/11 [==============================] - 20s 2s/step - loss: 4.5508e-04 - val_loss: 0.0070
Epoch 182/200
11/11 [==============================] - 20s 2s/step - loss: 4.4742e-04 - val_loss: 0.0064
Epoch 183/200
11/11 [==============================] - 19s 2s/step - loss: 4.1791e-04 - val_loss: 0.0067
Epoch 184/200
11/11 [==============================] - 20s 2s/step - loss: 5.8884e-04 - val_loss: 0.0065
Epoch 185/200
11/11 [==============================] - 19s 2s/step - loss: 6.1943e-04 - val_loss: 0.0064
Epoch 186/200
11/11 [==============================] - 20s 2s/step - loss: 5.7206e-04 - val_loss: 0.0069
Epoch 187/200
11/11 [==============================] - 19s 2s/step - loss: 6.5776e-04 - val_loss: 0.0066
Epoch 188/200
11/11 [==============================] - 19s 2s/step - 

In [29]:
model.save('./model/object_detection.h5')